The task is simple: find the best combination of pre-processing steps and predictive models with respect to an objective criterion. Logistically this can be problematic: a small example might involve three classification models, and two data preprocessing steps with two possible variations for each — overall 12 combinations. For each of these combinations we would like to perform a grid search of predefined hyperparameters on a fixed cross-validation dataset, computing performance metrics for each option (for example ROC AUC). Clearly this can become complicated quickly. On the other hand, many of these combinations share substeps, and re-running such shared steps amounts to a loss of compute time.

## 1. Defining Pipelines Steps and Grid Search Parameters

The researcher specifies the possible processing steps and the scikit objects involved, then Reskit expands these steps to each possible pipeline. Reskit represents these pipelines in a convenient pandas dataframe, so the researcher can directly visualize and manipulate the experiments.

In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA

from reskit.core import Pipeliner

# Feature selection and feature extraction step variants (1st step)
feature_engineering = [('VT', VarianceThreshold()),
                       ('PCA', PCA())]

# Preprocessing step variants (2nd step)
scalers = [('standard', StandardScaler()),
           ('minmax', MinMaxScaler())]

# Models (3rd step)
classifiers = [('LR', LogisticRegression()),
               ('SVC', SVC()),
               ('SGD', SGDClassifier())]

# Reskit needs to define steps in this manner
steps = [('feature_engineering', feature_engineering),
         ('scaler', scalers),
         ('classifier', classifiers)]

# Grid search parameters for our models
param_grid = {'LR': {'penalty': ['l1', 'l2']},
              'SVC': {'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
              'SGD': {'penalty': ['elasticnet'],
                      'l1_ratio': [0.1, 0.2, 0.3]}}

# Quality metric that we want to optimize
scoring='roc_auc'

pipe = Pipeliner(steps, param_grid=param_grid)
pipe.plan_table

,feature_engineering,scaler,classifier
0,VT,standard,LR
1,VT,standard,SVC
2,VT,standard,SGD
3,VT,minmax,LR
4,VT,minmax,SVC
5,VT,minmax,SGD
6,PCA,standard,LR
7,PCA,standard,SVC
8,PCA,standard,SGD
9,PCA,minmax,LR


## 2. Forbidden combinations

In case you don't want to use minmax scaler with SVC, you can define banned combo:

In [19]:
banned_combos = [('minmax', 'SVC')]
pipe = Pipeliner(steps, param_grid=param_grid, banned_combos=banned_combos)
pipe.plan_table

,feature_engineering,scaler,classifier
0,VT,standard,LR
1,VT,standard,SVC
2,VT,standard,SGD
3,VT,minmax,LR
4,VT,minmax,SGD
5,PCA,standard,LR
6,PCA,standard,SVC
7,PCA,standard,SGD
8,PCA,minmax,LR
9,PCA,minmax,SGD


## 3. Launching Experiment

Reskit then runs each experiment and presents results which are provided to the user through a pandas dataframe. For each pipeline’s classifier, Reskit grid search on cross-validation to find the best classifier’s parameters and report metric mean and standard deviation for each tested pipeline (ROC AUC in this case).

In [20]:
from sklearn.datasets import make_classification


X, y = make_classification()
pipe.get_results(X, y, scoring=['roc_auc'])

Line: 1/10
Line: 2/10
Line: 3/10
Line: 4/10
Line: 5/10
Line: 6/10
Line: 7/10
Line: 8/10
Line: 9/10
Line: 10/10


,feature_engineering,scaler,classifier,grid_roc_auc_mean,grid_roc_auc_std,grid_roc_auc_best_params,eval_roc_auc_mean,eval_roc_auc_std,eval_roc_auc_scores
0,VT,standard,LR,0.906544,0.0521184,{'penalty': 'l1'},0.90784,0.0524127,[ 0.84429066 0.90657439 0.97265625]
1,VT,standard,SVC,0.933529,0.024329,{'kernel': 'sigmoid'},0.93422,0.0245793,[ 0.92041522 0.91349481 0.96875 ]
2,VT,standard,SGD,0.889632,0.0634333,"{'l1_ratio': 0.3, 'penalty': 'elasticnet'}",0.873351,0.0847754,[ 0.75432526 0.92041522 0.9453125 ]
3,VT,minmax,LR,0.951397,0.019943,{'penalty': 'l1'},0.951967,0.0201537,[ 0.93771626 0.93771626 0.98046875]
4,VT,minmax,SGD,0.922132,0.0464637,"{'l1_ratio': 0.2, 'penalty': 'elasticnet'}",0.914761,0.0413263,[ 0.87889273 0.89273356 0.97265625]
5,PCA,standard,LR,0.849926,0.122804,{'penalty': 'l1'},0.85218,0.122634,[ 0.6816609 0.9100346 0.96484375]
6,PCA,standard,SVC,0.783235,0.0779878,{'kernel': 'rbf'},0.781052,0.0787479,[ 0.816609 0.85467128 0.671875 ]
7,PCA,standard,SGD,0.769632,0.180116,"{'l1_ratio': 0.2, 'penalty': 'elasticnet'}",0.761548,0.19374,[ 0.48788927 0.9100346 0.88671875]
8,PCA,minmax,LR,0.873456,0.0743371,{'penalty': 'l1'},0.875248,0.0746873,[ 0.78200692 0.87889273 0.96484375]
9,PCA,minmax,SGD,0.821544,0.124441,"{'l1_ratio': 0.3, 'penalty': 'elasticnet'}",0.824201,0.124639,[ 0.65743945 0.85813149 0.95703125]


## 4. Caching intermediate steps

Reskit also allows you to cache interim calculations to avoid unnecessary recalculations.

In [24]:
from sklearn.preprocessing import Binarizer

# Simple binarization step that we want ot cache
binarizer = [('binarizer', Binarizer())]

# Reskit needs to define steps in this manner
steps = [('binarizer', binarizer),
         ('classifier', classifiers)]

pipe = Pipeliner(steps, param_grid=param_grid)
pipe.plan_table

,binarizer,classifier
0,binarizer,LR
1,binarizer,SVC
2,binarizer,SGD


In [25]:
pipe.get_results(X, y, caching_steps=['binarizer'])

Line: 1/3
Line: 2/3
Line: 3/3


,binarizer,classifier,grid_accuracy_mean,grid_accuracy_std,grid_accuracy_best_params,eval_accuracy_mean,eval_accuracy_std,eval_accuracy_scores
0,binarizer,LR,0.84,0.0711461,{'penalty': 'l1'},0.841912,0.0717304,[ 0.76470588 0.82352941 0.9375 ]
1,binarizer,SVC,0.83,0.0747348,{'kernel': 'rbf'},0.832108,0.0754846,[ 0.76470588 0.79411765 0.9375 ]
2,binarizer,SGD,0.79,0.107526,"{'l1_ratio': 0.1, 'penalty': 'elasticnet'}",0.782475,0.0946448,[ 0.67647059 0.76470588 0.90625 ]


Last cached calculations stored in _cached_X

In [23]:
pipe._cached_X

OrderedDict([('init',
              array([[ 1.0949224 , -0.05378298, -0.95081274, ..., -1.21132477,
                      -0.56229831, -1.49348398],
                     [-1.26668126,  0.48509669, -0.06619828, ..., -0.03818197,
                      -0.07626735, -0.60853408],
                     [-1.75158649,  1.28625203,  0.92408772, ..., -0.67324595,
                       0.14502668,  0.82211376],
                     ..., 
                     [ 1.47338427, -1.24442992, -0.5457924 , ...,  0.04686373,
                      -0.21064959, -1.42582022],
                     [-1.61430948,  1.54696038, -0.32109691, ...,  1.07953423,
                       0.87835688,  0.65628271],
                     [-1.27347511, -2.44300832,  1.14676147, ..., -0.06189342,
                      -0.42794953,  2.24539721]])),
             ('binarizer', array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
                     [ 0.,  1.,  0., ...,  0.,  0.,  0.],
                     [ 0.,  1.,  1., ...,  0.,  1.,